# Data Collection and Cleaning

**Research Question: What factors effect a country's proportion of women in high ranking positions?**

We wanted to see of there was a correlation between women involvement in government and prevalence of women in the private sector. We hypothesized that more women in government will mean more women in leadership roles of companies. Female legislators could lead the country to increase laws concerning maternity leave, parental rights, contreception, divorce laws, etc. We believe that these changes, led by women leaders, would contribute to women being able to work full time and to be able to hold management positions. We were also interested to see what other factors would contribute to women in government so we examined the demographics of countries. 



We used many separate data sets and wanted to combine all of these data sets by country name. To address the problem of different sources using different styles and notaion to label the same country, we created two functions to help us combine data sets by country names. We also always double checked that the functions correcly map the country names. If necesary, we also hard coded any differences in country names that the function did not detect.

In [80]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [81]:
def get_diff_dict(list1, list2):
    diff = {}
    for i in range(0, len(list1)):
        for j in range(0, len(list2)):
            ratio = similar(list1[i],list2[j])
            if (ratio > 0.7) & (ratio != 1):
                diff[list1[i]] = list2[j]
    return diff

Women in National Parliaments is a csv file which contains the proportion of women in national parliaments in each country and by year. The years 1991-1996 had no data and the "Indicator Name" and "Indicator Code" were not real variables (repeated "proportion of women in national parliament" for all columns), which is why we discarded these variables. To distinguish from other data sets, we made all the years have "P" which stands for parliament.

In [118]:
import numpy as np
import pandas as pd

parliament = pd.read_csv("WomenNationalParliaments.csv")
parliament.drop(["Indicator Name", "Indicator Code", "1991","1992","1993",
                "1994","1995", "1996"],
               axis=1, inplace=True)
parliament = parliament[~parliament["Country Name"].str.contains("IDA", regex = True)]
for i in range(1990, 2019):
    oldname = str(i)
    newname = str(i) + "_P"
    parliament = parliament.rename(columns={oldname:newname})
parliament.loc[:,"1990_P":"2018_P"] = parliament.loc[:,"1990_P":"2018_P"].astype(float)
parliament.head()

,Country Name,Country Code,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2009_P,2010_P,2011_P,2012_P,2013_P,2014_P,2015_P,2016_P,2017_P,2018_P
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.3,27.7,27.7,27.7,27.7,27.7,27.7,27.7,27.7,27.7
2,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,38.6,38.6,38.2,34.1,34.1,36.8,36.8,36.8,38.2,30.5
3,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,16.4,16.4,15.7,15.7,17.9,20.0,20.7,22.9,27.9,27.9
4,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,35.7,35.7,50.0,50.0,50.0,50.0,39.3,32.1,32.1,32.1


Income groups contained the information about each countries income group, as compared to all other countries. There were a few unhelpful variables that we decided to drop.

In [83]:
income_groups = pd.read_csv("IncomeGroups.csv")
income_groups = income_groups.rename(columns={"TableName":"Country Name"})
income_groups.drop(["SpecialNotes", "Unnamed: 5"], axis=1, inplace=True)
income_groups = income_groups[~income_groups[
    "Country Name"].str.contains("IDA",regex = True)]
income_groups.head()

,Country Code,Region,IncomeGroup,Country Name
0,ABW,Latin America & Caribbean,High income,Aruba
1,AFG,South Asia,Low income,Afghanistan
2,AGO,Sub-Saharan Africa,Lower middle income,Angola
3,ALB,Europe & Central Asia,Upper middle income,Albania
4,AND,Europe & Central Asia,High income,Andorra


In [84]:
diff_income = list(set(list(income_groups["Country Name"].values)) - set(
        list(parliament["Country Name"].values)))
diff_parliament = list(set(list(parliament["Country Name"].values)) - set(
        list(income_groups["Country Name"].values)
    ))
diff = get_diff_dict(diff_income,diff_parliament)
diff

{"Côte d'Ivoire": "Cote d'Ivoire",
 "Korea, Dem. People's Rep.": 'Korea, Dem. People’s Rep.',
 'São Tomé and Principe': 'Sao Tome and Principe',
 'Curaçao': 'Curacao'}

In [85]:
income_groups = income_groups.set_index("Country Name")
income_groups.rename(index= diff,inplace=True)

In [86]:
# merge parliament and income data sets
parliament_income = parliament.merge(income_groups, on = "Country Name", 
                                how = "outer", suffixes=('', '_y')
                                  ).drop("Country Code_y", axis = 1)
parliament_income.head()

,Country Name,Country Code,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2011_P,2012_P,2013_P,2014_P,2015_P,2016_P,2017_P,2018_P,Region,IncomeGroup
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latin America & Caribbean,High income
1,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.7,27.7,27.7,27.7,27.7,27.7,27.7,27.7,South Asia,Low income
2,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,38.2,34.1,34.1,36.8,36.8,36.8,38.2,30.5,Sub-Saharan Africa,Lower middle income
3,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,15.7,15.7,17.9,20.0,20.7,22.9,27.9,27.9,Europe & Central Asia,Upper middle income
4,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,50.0,50.0,50.0,50.0,39.3,32.1,32.1,32.1,Europe & Central Asia,High income


In [87]:
#dropping all non-countries while manually checking that it didn't drop any actual countries
parliament_income = parliament_income[
    ~parliament_income["Country Name"].str.contains(
    "income|area|IDA|only|Asia|Europe|Latin|North|"
    "Saharan|World|dividend|OECD|UN|affected|HIPC|classified",
    regex = True)]

Similarly to parliament, we wanted to distinguish the variables from this data set that were originally just named the year, so we added an "M" for management. This csv gives us the proportion of women in senior and middle management positions in each country. We also got rid of columns that are not of intrest to our research question. 

In [88]:
management = pd.read_csv("WomenSeniorMiddleManagementPositions.csv")
management = management.drop(["geoAreaCode", "series_release", 
                "series_code", "series_description",  "X", "Y", 
                "sliceId", "Sex", "Units", "Sex_description", 
                "latest_year","Units_description", "latest_source", 
                "latest_nature", "last_5_years_mean", "FID"], axis = 1)
management = management.rename(columns={"geoAreaName":"Country Name", 
                                      "ISO3CD":"Country Code", "latest_value":"latest_value_M"})
management = management.set_index("Country Name")
for i in range(2000, 2017):
    oldname = "F" + str(i)
    newname = str(i) + "_M"
    management = management.rename(columns={oldname:newname})
management.loc[:,"2000_M":"2016_M"] = management.loc[:,"2000_M":"2016_M"].astype(float)
management.head()

,Country Code,2000_M,2001_M,2002_M,2003_M,2004_M,2005_M,2006_M,2007_M,2008_M,2009_M,2010_M,2011_M,2012_M,2013_M,2014_M,2015_M,2016_M,latest_value_M
Country Name,,,,,,,,,,,,,,,,,,,
Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Albania,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.47
Antarctica,ATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Algeria,DZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.92,NaN,NaN,6.92
American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
diff_mang = list(
    set(
        list(management.index.values)
    ) - set(
        list(parliament_income["Country Name"].values)
    ))
diff_parl_income = list(
    set(
        list(parliament_income["Country Name"].values)
    ) - set(
        list(management.index.values)
    ))
diff = get_diff_dict(diff_mang,diff_parl_income)
diff

{'Saint Martin (French Part)': 'St. Martin (French part)',
 'Saint Lucia': 'St. Lucia',
 'Gambia': 'Gambia, The',
 'Curaçao': 'Curacao',
 'Micronesia, Federated States of': 'Micronesia, Fed. Sts.',
 'Iran (Islamic Republic of)': 'Iran, Islamic Rep.',
 'Bahamas': 'Bahamas, The',
 "Côte d'Ivoire": "Cote d'Ivoire",
 'United States of America': 'United States',
 'Syrian Arab Republic (Syria)': 'Syrian Arab Republic',
 'Viet Nam': 'Vietnam',
 'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
 'Saint Kitts and Nevis': 'St. Kitts and Nevis',
 'Åland Islands': 'Channel Islands'}

In [90]:
del diff['Åland Islands']
management = management.rename(index=diff)

In [91]:
#there are still difference between the countries that can be fixed with hard coding the names
management = management.rename(index=
    {'China, Hong Kong Special Administrative Region': 'Hong Kong SAR, China',
    'China, Macao Special Administrative Region': 'Macao SAR, China',
    'State of Palestine': 'West Bank and Gaza',
    'Congo': 'Congo, Rep.',
    'Tanzania, United Republic of': 'Tanzania',
    'Republic of Moldova': 'Moldova',
    'Kyrgyzstan': 'Kyrgyz Republic',
    "Lao People's Democratic Republic": 'Lao PDR',
    'Venezuela (Bolivarian Republic)': 'Venezuela, RB',
    'Czechia': 'Czech Republic',
    'Slovakia': 'Slovak Republic',
    "Democratic People's Republic of Korea": 'Korea, Dem. People’s Rep.',
    'Egypt': 'Egypt, Arab Rep.',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Macedonia, Republic of': 'Macedonia, FYR',
    'Republic of Korea': 'Korea, Rep.',
    'Yemen': 'Yemen, Rep.',
    'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
    'United States Virgin Islands': 'Virgin Islands (U.S.)'})

In [92]:
# merge management dataset with previously merged parliament and incomes data sets
par_income_manag = parliament_income.merge(management, 
         on = "Country Name", how = "inner")
par_income_manag.head()

,Country Name,Country Code_x,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2008_M,2009_M,2010_M,2011_M,2012_M,2013_M,2014_M,2015_M,2016_M,latest_value_M
0,Aruba,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,NaN,22.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.47
4,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Scarping from wiki for the types of government in each country: 

In [93]:
import requests
from bs4 import BeautifulSoup
resp = requests.get("https://en.wikipedia.org/wiki/"
                    "List_of_countries_by_system_of_government")
soup = BeautifulSoup(resp.content, "html.parser")

In [94]:
tables = soup.find_all("table", {"class": "wikitable"})
recognized_countries = tables[0]

In [95]:
countries = recognized_countries.find_all("td")
rows = []
i = 0
while i < len(countries):
    country = str(countries[i]).split("title=")[1]
    start = country.find(">") + 1
    end = country.find("<") 
    country = country[start:end]
    gov_type = str(countries[i+1]).split("<td>")[1].strip("\n</td>")
    rows.append({
        "Country Name": country,
        "Government Type": gov_type
    })
    i+=4
gov_types = pd.DataFrame(rows)
gov_types["Government Type"]= gov_types["Government Type"].map(
                {'Republic':'Republic',
                'Constitutional\xa0monarchy': 'Constitutional Monarchy',
                'Absolute monarchy' : 'Absolute Monarchy'})
gov_types.head()

,Country Name,Government Type
0,Afghanistan,Republic
1,Albania,Republic
2,Algeria,Republic
3,Andorra,Constitutional Monarchy
4,Angola,Republic


In [96]:
diff_par = list(set(list(par_income_manag["Country Name"].values)) - set(
        list(gov_types["Country Name"].values)
    ))
diff_gov = list(set(list(gov_types["Country Name"].values)) - set
    (list(par_income_manag["Country Name"].values)
    ))
diff = get_diff_dict(diff_par, diff_gov)
diff

{'St. Kitts and Nevis': 'Saint Kitts and Nevis',
 'Venezuela, RB': 'Venezuela',
 'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
 'Sao Tome and Principe': 'São Tomé and Príncipe',
 "Cote d'Ivoire": "Côte d'Ivoire",
 'Cabo Verde': 'Cape Verde',
 'St. Lucia': 'Saint Lucia'}

In [97]:
par_income_manag = par_income_manag.set_index ("Country Name")
par_income_manag = par_income_manag.rename(index = diff)

In [98]:
set(list(gov_types["Country Name"].values)) - set(
    list(par_income_manag.index.values))

{'Brunei',
 "China, People's Republic of",
 'Congo, Democratic Republic of the',
 'Congo, Republic of the',
 'East Timor',
 'Egypt',
 'Eswatini',
 'Federated States of Micronesia',
 'Iran',
 'Korea, North',
 'Korea, South',
 'Kyrgyzstan',
 'Laos',
 'North Macedonia',
 'Palestine',
 'Russia',
 'Slovakia',
 'Syria',
 'Vatican City',
 'Yemen'}

In [99]:
# fixing more country names manually
par_income_manag = par_income_manag.rename(index = {
 'China': "China, People's Republic of",
 'Egypt, Arab Rep.':'Egypt',
 'Iran, Islamic Rep.': 'Iran',
 'Korea, Dem. People’s Rep.' : 'Korea, North',
 'Kyrgyz Republic':'Kyrgyzstan' ,
 'Lao PDR' : 'Laos',
 'Micronesia, Fed. Sts.' : 'Federated States of Micronesia',
 'Russian Federation': 'Russia',
 'Syrian Arab Republic' : 'Syria',
 'West Bank and Gaza' : 'Palestine'})

In [100]:
# merge data sets
par_gov = par_income_manag.merge(gov_types, on = "Country Name", how = "inner",)
par_gov.head()

,Country Name,Country Code_x,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2009_M,2010_M,2011_M,2012_M,2013_M,2014_M,2015_M,2016_M,latest_value_M,Government Type
0,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Republic
1,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Republic
2,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,22.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.47,Republic
3,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Constitutional Monarchy
4,United Arab Emirates,ARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Absolute Monarchy


This data set contained inforamtion about the religious groups of each country. For our interests, it was more benefitial to simply have the majority religion listed for each country, instead of having the percent of many different religions for each country. Thus, we dropped many unnecesary variables in the data set and created a column which only hold the majority religion in it. 

In [119]:
religion_df = pd.read_csv("religions.csv")
religion_df.head()

,row_number,level,Nation_fk,Year,Region,Country,All Religions,Buddhists,Christians,Folk Religions,Hindus,Jews,Muslims,Other Religions,Unaffiliated,Unnamed: 15
0,1.0,3.0,10000.0,2010.0,World,All Countries,100,7.1,31.4,5.9,15.0,< 1.0,23.2,< 1.0,16.4,NaN
1,2.0,2.0,1001.0,2010.0,North America,All Countries,100,1.1,77.4,< 1.0,< 1.0,1.8,1.0,< 1.0,17.1,NaN
2,3.0,2.0,1002.0,2010.0,Latin America-Caribbean,All Countries,100,< 1.0,90.0,1.7,< 1.0,< 1.0,< 1.0,< 1.0,7.7,NaN
3,4.0,2.0,1003.0,2010.0,Europe,All Countries,****,< 1.0,74.5,< 1.0,< 1.0,< 1.0,5.9,< 1.0,18.8,NaN
4,5.0,2.0,1004.0,2010.0,Middle East-North Africa,All Countries,100,< 1.0,3.7,< 1.0,< 1.0,1.6,93.0,< 1.0,< 1.0,NaN


In [102]:
religion_df = religion_df.drop(["row_number", "level", "Unnamed: 15"], axis=1)
religion_df = religion_df.set_index("Country")
religion_df = religion_df.replace(["< 1.0", ">99.0"], [0,100])
cols = ["Buddhists", "Christians", "Folk Religions", "Hindus",
        "Jews", "Muslims", "Other Religions", "Unaffiliated"]
religion_df[cols] = religion_df[cols].apply(pd.to_numeric, errors='coerce')
religion_df["Religion"] = religion_df[["Buddhists", "Christians",
                            "Folk Religions", "Hindus", "Jews", 
                            "Muslims", "Other Religions", 
                                        "Unaffiliated"]].idxmax(axis=1)
religion_df = religion_df.drop(' All Countries')

In [103]:
religion = religion_df.pivot(columns = "Year", values = "Religion")
country_religion = pd.DataFrame()
country_religion = religion[2010.0]
country_religions = pd.DataFrame(country_religion)
country_religions = country_religions.dropna()
country_religions = country_religions.rename(columns = {2010.0:"Religion"})
country_religions.head()

,Religion
Country,
Afghanistan,Muslims
Albania,Muslims
Algeria,Muslims
American Samoa,Christians
Andorra,Christians


The following data set contained demographic information about each country in the data set. We kept only the variables that we thought would be useful to us in our project. 

In [104]:
country_stats_df = pd.read_csv("world stats.csv")
country_stats_df = country_stats_df.drop(["Climate",
                    "Arable (%)", "Crops (%)", "Other (%)",
                    "Coastline (coast/area ratio)", 
                    "Net migration", "Phones (per 1000)", 
                    "Agriculture", "Industry", "Service",
                    "Region"], axis=1)
cs_idx = country_stats_df.Country.values
cs_idx = [c[:-1] for c in cs_idx]
country_stats_df["Country"] = cs_idx
country_stats_df = country_stats_df.set_index("Country")
#population is less informative than popoulation density, so we dropped it
country_stats_df = country_stats_df.drop(["Population", 
                            "Area (sq. mi.)"], axis=1)
country_stats_df.loc['Palestinian territories'] = (
    country_stats_df.loc["Gaza Strip"] + country_stats_df.loc["West Bank"])
country_stats_df = country_stats_df.drop(
    ["Gaza Strip", "West Bank"])
country_stats_df.head()

,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Birthrate,Deathrate
Country,,,,,,
Afghanistan,48.0,163.07,700.0,36.0,46.60,20.34
Albania,124.6,21.52,4500.0,86.5,15.11,5.22
Algeria,13.8,31.00,6000.0,70.0,17.14,4.61
American Samoa,290.4,9.27,8000.0,97.0,22.46,3.27
Andorra,152.1,4.05,19000.0,100.0,8.71,6.25


In [105]:
# merge the previous two data sets
diff_relig = list(
    set(list(country_religions.index.values)) - set(
        list(country_stats_df.index.values)))
diff_stat = list(
    set(list(country_stats_df.index.values)) - set(
        list(country_religions.index.values)))
diff = get_diff_dict(diff_stat, diff_relig)
diff 

{'Saint Lucia': 'St. Lucia',
 'Trinidad & Tobago': 'Trinidad and Tobago',
 'Bahamas. The': 'Bahamas',
 'British Virgin Is.': 'British Virgin Islands',
 'Antigua & Barbuda': 'Antigua and Barbuda',
 'Saint Helena': 'St. Helena',
 'St Pierre & Miquelon': 'St. Pierre and Miquelon',
 'N. Mariana Islands': 'U.S. Virgin Islands',
 'Saint Kitts & Nevis': 'St. Kitts and Nevis',
 'Central African Rep.': 'Central African Republic',
 'Sao Tome & Principe': 'Sao Tome and Principe',
 'Bosnia & Herzegovina': 'Bosnia-Herzegovina',
 'Turks & Caicos Is': 'Turks and Caicos Islands',
 'Virgin Islands': 'U.S. Virgin Islands',
 'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
 'Gambia. The': 'Gambia',
 'Faroe Islands': 'Channel Islands'}

In [106]:
del diff['N. Mariana Islands']
del diff['Faroe Islands']

In [107]:
country_stats_df = country_stats_df.rename(index=diff)

In [108]:
# more manual corrections
country_stats_df = country_stats_df.rename(index={
 'Congo, Repub. of the': 'Republic of the Congo',
 'Faroe Islands': 'Faeroe Islands',
 'Congo, Dem. Rep.': 'Democratic Republic of the Congo',
 'Korea, North': 'North Korea',
 'Macedonia': 'Republic of Macedonia',
 'Burma': 'Burma (Myanmar)',
 'Korea, South': 'South Korea'})

In [109]:
stats_religion = pd.merge(country_religions, 
                          country_stats_df, on = "Country",
                          how = "inner", 
                          suffixes=('', '_y'))
stats_religion.head()

,Religion,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Birthrate,Deathrate
Country,,,,,,,
Afghanistan,Muslims,48.0,163.07,700.0,36.0,46.60,20.34
Albania,Muslims,124.6,21.52,4500.0,86.5,15.11,5.22
Algeria,Muslims,13.8,31.00,6000.0,70.0,17.14,4.61
American Samoa,Christians,290.4,9.27,8000.0,97.0,22.46,3.27
Andorra,Christians,152.1,4.05,19000.0,100.0,8.71,6.25


In [110]:
# merge final two data sets
diff_relig = list(
    set(list(stats_religion.index.values)) - set(
        list(par_gov["Country Name"].values)))
diff_par = list(
    set(list(par_gov["Country Name"].values)) - set(
        list(stats_religion.index.values)))
diff = get_diff_dict(diff_relig, diff_par)
diff

{'Gambia': 'Gambia, The',
 'St. Lucia': 'Saint Lucia',
 'St. Kitts and Nevis': 'Saint Kitts and Nevis',
 'Bahamas': 'Bahamas, The',
 'St. Vincent and the Grenadines': 'Saint Vincent and the Grenadines',
 'Sao Tome and Principe': 'São Tomé and Príncipe',
 'Bosnia-Herzegovina': 'Bosnia and Herzegovina'}

In [111]:
stats_religion = stats_religion.rename(index = diff)

In [112]:
set(list(par_gov["Country Name"].values)) - set(
    list(stats_religion.index.values))

{"China, People's Republic of",
 "Côte d'Ivoire",
 'Federated States of Micronesia',
 'Korea, North',
 'Montenegro',
 'Myanmar',
 'Palestine',
 'South Sudan'}

In [113]:
#manual corrections
stats_religion = stats_religion.rename(
    index={
        'Micronesia, Fed. St.': 'Federated States of Micronesia',
        'China': "China, People's Republic of",
        'North Korea':'Korea, North',
        'South Korea':'Korea, South',
        'Palestinian territories':'Palestine',
        'Burma (Myanmar)': 'Myanmar'})

In [114]:
par_gov = par_gov.rename(columns = {"Country Name": "Country"})
par_gov.head()

,Country,Country Code_x,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2009_M,2010_M,2011_M,2012_M,2013_M,2014_M,2015_M,2016_M,latest_value_M,Government Type
0,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Republic
1,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Republic
2,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,22.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.47,Republic
3,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Constitutional Monarchy
4,United Arab Emirates,ARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Absolute Monarchy


Our final data frame includes proportion of women in national parliaments by year, proportion of women in senior and middle management positions by year, income groups of each country, most common religion and government types, and demographic information about the countries. 

In [115]:
final = par_gov.merge(stats_religion, on = "Country", how = "inner")
final = final.rename(columns = {"Country Code_x": "Country Code"}
                    ).drop("Country Code_y", axis = 1)
final.head()

,Country,Country Code,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2016_M,latest_value_M,Government Type,Religion,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Birthrate,Deathrate
0,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Republic,Muslims,48.0,163.07,700.0,36.0,46.60,20.34
1,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,NaN,NaN,Republic,Christians,9.7,191.19,1900.0,42.0,45.11,24.20
2,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,NaN,22.47,Republic,Muslims,124.6,21.52,4500.0,86.5,15.11,5.22
3,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,NaN,NaN,Constitutional Monarchy,Christians,152.1,4.05,19000.0,100.0,8.71,6.25
4,United Arab Emirates,ARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,Absolute Monarchy,Muslims,31.4,14.51,23200.0,77.9,18.96,4.40


In [116]:
final.to_csv("~/data301/share/Brooke_Michal/final_data.csv", index = False)

In [117]:
final.head()

,Country,Country Code,1990_P,1997_P,1998_P,1999_P,2000_P,2001_P,2002_P,2003_P,...,2016_M,latest_value_M,Government Type,Religion,Pop. Density (per sq. mi.),Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Birthrate,Deathrate
0,Afghanistan,AFG,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Republic,Muslims,48.0,163.07,700.0,36.0,46.60,20.34
1,Angola,AGO,14.5,9.5,15.5,15.5,15.5,15.5,15.5,15.5,...,NaN,NaN,Republic,Christians,9.7,191.19,1900.0,42.0,45.11,24.20
2,Albania,ALB,28.8,NaN,NaN,5.2,5.2,5.7,5.7,5.7,...,NaN,22.47,Republic,Muslims,124.6,21.52,4500.0,86.5,15.11,5.22
3,Andorra,AND,NaN,7.1,7.1,7.1,7.1,14.3,14.3,14.3,...,NaN,NaN,Constitutional Monarchy,Christians,152.1,4.05,19000.0,100.0,8.71,6.25
4,United Arab Emirates,ARE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,Absolute Monarchy,Muslims,31.4,14.51,23200.0,77.9,18.96,4.40


In [120]:
len(final)

180